In [1]:
import pandas as pd

df = pd.read_csv('../input/melbourne-housing/melb_data.csv')
df.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [2]:
df.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [5]:
y = df.Price
X = df.drop(['Price'], axis=1).select_dtypes(exclude=['object'])

In [8]:
X.describe()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score(X_train, y_train, X_valid, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    predicts = model.predict(X_valid)
    return mean_absolute_error(y_valid, predicts)

In [21]:
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score(reduced_X_train, y_train, reduced_X_valid, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


In [23]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

print("MAE from Approach 2 (Simple Imputation):")
print(score(imputed_X_train, y_train, imputed_X_valid, y_valid))

MAE from Approach 2 (Simple Imputation):
178166.46269899711


In [24]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for c in cols_with_missing:
    X_train_plus[c + "_missing_value"] = X_train_plus[c].isnull().any()
    X_valid_plus[c + "_missing_value"] = X_valid_plus[c].isnull().any()
    
simple_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(simple_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(simple_imputer.transform(X_valid_plus))

print("MAE from Approach 2 (Simple Imputation with columns):")
print(score(imputed_X_train_plus, y_train, imputed_X_valid_plus, y_valid))

MAE from Approach 2 (Simple Imputation with columns):
177694.17855564906


In [25]:
cols_with_missing

['Car', 'BuildingArea', 'YearBuilt']

In [27]:
X_train.describe()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,10864.000000,10864.000000,10864.000000,10864.000000,10864.000000,10815.000000,10864.000000,5708.000000,6557.000000,10864.000000,10864.000000,10864.000000
mean,2.941458,10.123518,3105.655099,2.916973,1.535162,1.604716,565.872423,153.764119,1964.839866,-37.809911,144.995053,7474.990519
std,0.961697,5.881645,91.169384,0.973700,0.692709,0.961010,4382.784066,601.749174,36.396266,0.079467,0.103625,4359.830215
min,1.000000,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1830.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.100000,3046.000000,2.000000,1.000000,1.000000,178.000000,93.000000,1940.000000,-37.857300,144.929667,4385.000000
50%,3.000000,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802465,145.000200,6567.000000
75%,4.000000,13.000000,3149.000000,3.000000,2.000000,2.000000,650.250000,174.000000,2000.000000,-37.757172,145.057518,10331.000000
max,10.000000,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.482730,21650.000000


In [28]:
print(X_train.shape)

(10864, 12)


In [34]:
X_train_isnull = X_train.isnull().sum()
X_train_isnull[X_train_isnull > 0]

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64